In [ ]:
Input Data

In [ ]:
from sklearn.datasets import fetch_lfw_people

def dataGetter():
    # Fetch the Data
    lfw_people = fetch_lfw_people(min_faces_per_person = 70, resize = 0.4)
    X = lfw_people.data
    return(X)
    
def nextBatch(X, size):
    # Get the random indexes as per the size
    idx = np.random.choice(np.arange(len(X)), size, replace = False)
    x_sample = X[idx]
    return(x_sample)


# Total Retreived Data
dataX = dataGetter()

# Train Data
train_X = dataX[0:1281]

# Testing Data
test_X = dataX[1281:1289]

In [ ]:
Deep Autoencoder Network

In [ ]:
import tensorflow as tf
from functools import partial

# Reset the graph and create saver
tf.reset_default_graph()

# Initialize the parameters
n_inputs = 50*37
n_hidden1 = 1800
n_hidden2 = 1750
n_hidden3 = 1700
n_hidden4 = 1650
n_hidden5 = n_hidden3
n_hidden6 = n_hidden2
n_hidden7 = n_hidden1
n_outputs = n_inputs
learning_rate = 0.001
l2_reg = 0.01

# Placeholder to keep the input data
X = tf.placeholder(tf.float32, shape = [None, n_inputs])

X_noisy = X + tf.random_normal(tf.shape(X))

# Adding the Layers
he_init = tf.contrib.layers.variance_scaling_initializer()
l2_regularizer = tf.contrib.layers.l2_regularizer(l2_reg)
my_dense_layer = partial(tf.layers.dense, activation = tf.nn.relu, kernel_initializer = he_init,
                        kernel_regularizer = l2_regularizer)
hidden1 = my_dense_layer(X_noisy, n_hidden1)
hidden2 = my_dense_layer(hidden1, n_hidden2)
hidden3 = my_dense_layer(hidden2, n_hidden3)
hidden4 = my_dense_layer(hidden3, n_hidden4)
hidden5 = my_dense_layer(hidden4, n_hidden5)
outputs = my_dense_layer(hidden5, n_outputs, activation = None)

# Calculating Loss and doing optimization
reconstruction_loss = tf.reduce_mean(tf.square(outputs - X))
reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
loss = tf.add_n([reconstruction_loss] + reg_losses)
optimizer = tf.train.AdamOptimizer(learning_rate)
training_op = optimizer.minimize(loss)

# Executing the Computational Graph
init = tf.global_variables_initializer()

saver = tf.train.Saver()

n_epochs = 12000
batch_size = 1200
n_batches = 1
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(n_batches):
            X_batch = nextBatch(train_X, batch_size)
            sess.run(training_op, feed_dict = {X: X_batch})
            loss_train = reconstruction_loss.eval(feed_dict = {X: X_batch})
        print(epoch, "\tMSE: ", loss_train)
    saver.save(sess, "./my_model_all_layers.ckpt")


In [ ]:
Plotting the Input and Ouput Data

In [ ]:
n_digits = 2

# To plot figures
% matplotlib inline
import matplotlib.pyplot as plt

plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

def plot_image(image, shape = [28, 28]):
    plt.imshow(image.reshape(shape), cmap = "Greys", interpolation = "nearest")
    plt.axis("off")
    
with tf.Session() as sess:
    saver.restore(sess, "./my_model_all_layers.ckpt")
    X_test = mnist.test.images[:n_digits]
    outputs_val = outputs.eval(feed_dict = {X :X_test})
    fig = plt.figure(figsize = (8,6))
    for index in range(n_digits):
        plt.subplot(n_digits, 2, index *2 +1)
        plot_image(X_test[index])
        plt.subplot(n_digits, 2, index*2+2)
        plot_image(outputs_val[index])